In [8]:
import random
from deap import base, creator, tools, algorithms
import cv2
import numpy as np
from knn_torch_upscaler import scale_up_pytorch
import torch

In [9]:
image_path = 'result.png'
radius = 4
scale = 1.414213562
target_scale = 2.0

In [10]:
import os

target_images = []
images_lr = []

folder = './optimizer_target_images'
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    image = cv2.imread(path)
    h, w = image.shape[:2]
    target_images.append(image)
    images_lr.append(cv2.resize(image, (int(w / target_scale+0.5), int(h / target_scale+0.5)), interpolation=cv2.INTER_AREA))

In [11]:
# Ziel: Maximierung (1.0 gibt an, dass es sich um eine Maximierungsaufgabe handelt)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def evalFunction(individual):
    a,b,c,d,e,r,t,u,k,s,v,w,z = individual
    k = int(k)
    if k%2==0:
        k-=1
    img_idx = random.randint(0,6)
    input_img = images_lr[img_idx]
    for i in range(2):
        h, w = input_img.shape[:2]
        input_img = scale_up_pytorch(input_img,int(w*scale+0.5),int(h*scale+0.5),torch.device("cuda"),a,b,c,d,e,int(r),int(t),u,k,s,v,w,z)
    diff = np.abs(input_img.astype(np.int32)-target_images[img_idx].astype(np.int32)).sum()
    return (-diff,)

toolbox = base.Toolbox()
toolbox.register("attr_a", random.uniform, 1, 14)
toolbox.register("attr_b", random.uniform, 1, 10)
toolbox.register("attr_c", random.uniform, 0.1, 3)
toolbox.register("attr_d", random.uniform, 0.1, 5)
toolbox.register("attr_e", random.uniform, 0.1, 0.8)
toolbox.register("attr_r", random.randint, 2, 4)
toolbox.register("attr_t", random.randint, 1, 6)
toolbox.register("attr_u", random.uniform, 0.1, 12)
toolbox.register("attr_k", random.randint, 7, 8)
toolbox.register("attr_s", random.uniform, 1.1, 3)
toolbox.register("attr_v", random.uniform, 0, 10)
toolbox.register("attr_w", random.randint, 0, 30)
toolbox.register("attr_z", random.uniform, 0, 3)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_a, toolbox.attr_b, toolbox.attr_c, toolbox.attr_d,
                  toolbox.attr_e,toolbox.attr_r,toolbox.attr_t,toolbox.attr_u,
                  toolbox.attr_k,toolbox.attr_s,toolbox.attr_v,toolbox.attr_w,toolbox.attr_z), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalFunction)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutPolynomialBounded, low=[1, 1, 0.1, 0.1,0.1,3,1,0.1,7,1.1,0,0,0], up=[30, 10, 3, 10,0.8,4,6,12,8,3,10,30,3], indpb=0.2, eta=1.0)
toolbox.register("select", tools.selTournament, tournsize=3)

/home/jan/.miniconda3/envs/upNN/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/jan/.miniconda3/envs/upNN/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
#Initialisierung der Population

pop = toolbox.population(n=3000)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", lambda x: sum([y[0] for y in x]) / len(x))
stats.register("min", lambda x: min([y[0] for y in x]))
stats.register("max", lambda x: max([y[0] for y in x]))

# Evolutionärer Prozess
algorithms.eaMuPlusLambda(pop, toolbox, mu=333, lambda_=777, cxpb=0.5, mutpb=0.2, ngen=137, stats=stats, halloffame=hof, verbose=True)

print("Best individual is: ", hof[0])

gen	nevals	avg         	min         	max    
0  	3000  	-8.08925e+06	-1.71961e+07	-320192
1  	550   	-3.75661e+06	-1.35379e+07	-322861
2  	544   	-1.74354e+06	-7.91448e+06	-323819
3  	556   	-1.08404e+06	-4.50927e+06	-321151
4  	551   	-784636     	-2.17237e+06	-321151
5  	550   	-628416     	-1.65727e+06	-321151
6  	551   	-540413     	-1.54715e+06	-318835
7  	542   	-464246     	-1.56635e+06	-318835
8  	549   	-415372     	-1.23686e+06	-318835
9  	559   	-399535     	-1.61295e+06	-317597
10 	547   	-397539     	-1.63168e+06	-317597
11 	551   	-378098     	-1.54652e+06	-316559
12 	546   	-382028     	-1.41689e+06	-317445
13 	568   	-380687     	-1.41755e+06	-316314
14 	568   	-387742     	-1.41707e+06	-314688
15 	525   	-372204     	-1.4149e+06 	-314688
16 	532   	-368534     	-2.03466e+06	-314688
17 	534   	-352619     	-1.19827e+06	-314688
18 	519   	-366156     	-1.41633e+06	-312264
19 	534   	-369451     	-1.62874e+06	-311915
20 	534   	-355634     	-1.41633e+06	-311915
21 	563   

: 